In [1]:
import random

import cv2
import numpy as np

Download images from directory.

In [2]:
from image import read_image, show_image

In [3]:
images, image_names = read_image("InputImages")

In [4]:
image_num = 0
image = images[image_num]
image_names[image_num]

'1633601732_6-p-foto-doma-snizu-vverkh-foto-6.jpg'

In [5]:
show_image("OutputImage", image)

# Stratification

Get lines from the image.

In [8]:
from line import get_lines, draw_line

In [9]:
lines = get_lines(image)
len(lines)

40

Plot lines.

In [10]:
ouput_image = image.copy()
# Getting the lines form the image
for line in lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(ouput_image, line, (r, g, b))
show_image("OutputImage", ouput_image)

Cluster lines to find vanishing points.

In [11]:
from clustering import cluster_lines

In [19]:
n_clusters = 4

In [20]:
clusters = cluster_lines(lines, n_clusters=n_clusters)
clusters

defaultdict(list,
            {1: [array([[443.        ,   0.75049156]], dtype=float32),
              array([[261.       ,   0.6981317]], dtype=float32),
              array([[245.       ,   0.6806784]], dtype=float32),
              array([[569.       ,   0.7853982]], dtype=float32),
              array([[361.      ,   0.715585]], dtype=float32),
              array([[260.       ,   0.6806784]], dtype=float32),
              array([[363.       ,   0.7330383]], dtype=float32),
              array([[306.      ,   0.715585]], dtype=float32),
              array([[585.       ,   0.7853982]], dtype=float32),
              array([[532.       ,   0.7853982]], dtype=float32),
              array([[303.       ,   0.6981317]], dtype=float32),
              array([[254.       ,   0.6981317]], dtype=float32)],
             2: [array([[-34.       ,   2.2340214]], dtype=float32),
              array([[-92.       ,   2.2514746]], dtype=float32),
              array([[40.       ,  2.2165682]], dtype

Show image with clustered lines and find vanishing points.

In [21]:
from homography import get_vanishing_point, get_nearest_points

In [22]:
vanishing_points = [get_vanishing_point(clusters[c]) for c in clusters]
vanishing_points

[array([-18528.059, -12603.677], dtype=float32),
 array([-1668.4064,  8764.042 ], dtype=float32),
 array([-1087.7777,  -159.3853], dtype=float32),
 array([2662628.8 ,  256002.77], dtype=float32)]

In [23]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = image.copy()
for c in clusters:
    vp = vanishing_points[c]
    for i, line in enumerate(clusters[c]):
        draw_line(output_image, line, colors[c])
    cv2.circle(output_image, (int(vp[0]), int(vp[1])), 10, colors[c], -1)

show_image("OutputImage", output_image) 

In [24]:
two_points, _ = get_nearest_points(vanishing_points)
two_points

[array([-1668.4064,  8764.042 ], dtype=float32),
 array([-1087.7777,  -159.3853], dtype=float32)]

In [25]:
point = [*max(two_points, key = lambda x: np.linalg.norm(x)), 1]
point

[-1668.4064, 8764.042, 1]

In [27]:
point = [*vanishing_points[2], 1]
point

[-1087.7777, -159.3853, 1]

# Test vanishing points search

In [28]:
from line import get_intersection_point, get_homogeneous_coordinates

In [29]:
output_image = image.copy()
for c in clusters:
    vp = vanishing_points[c]
    for i, line in enumerate(clusters[c][:2]):
        draw_line(output_image, line, colors[c])
    print(get_intersection_point(*clusters[c][:2]), get_homogeneous_coordinates(clusters[c][0]), get_homogeneous_coordinates(clusters[c][1]))
    show_image("OutputImage", output_image)

[-2039.7706, 2836.9482] [0.7313537, 0.6819983, -443.0] [0.76604444, 0.6427876, -261.0]
[2639.9976, 2019.4453] [-0.6156615, 0.7880108, 34.0] [-0.6293203, 0.77714604, 92.0]
[256.99652, -310.0723] [-0.9945219, 0.104528494, 288.0] [-0.9975641, 0.06975646, 278.0]


/Users/michael/Documents/IITP/Homography/line.py:80: RuntimeWarning: divide by zero encountered in scalar divide
  return [a / c, b / c]


[-inf, inf] [0.9998477, 0.017452406, -239.0] [0.9998477, 0.017452406, -247.0]


# Estimate intrinsic camera properties

In [6]:
width, height, _ = image.shape

In [7]:
diagonal = np.sqrt(width ** 2 + height ** 2)

In [8]:
f_x = f_y = 0.75 * diagonal

In [9]:
c_x = width / 2
c_y = width / 2

In [10]:
K = np.array([
    [f_x, 0, c_x],
    [0, f_y, c_y],
    [0, 0, 1],
])

# Get rotation in 3d world coordinates

In [12]:
point = np.linalg.inv(K) @ point
point

array([ 0.10877302, -0.16328419,  1.        ])

In [13]:
target_point = [0, -1, 0]

In [14]:
# target_point = [1, 0, 0]

In [15]:
import math

In [16]:
def rotate_along_x(point: np.ndarray, target_point: np.ndarray) -> np.ndarray:
    theta_x = math.atan2(target_point[2], target_point[1]) - math.atan2(point[2], point[1])
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(theta_x), -np.sin(theta_x)],
        [0, np.sin(theta_x), np.cos(theta_x)],
    ])
    
    return R_x

In [17]:
def rotate_along_y(point: np.ndarray, target_point: np.ndarray) -> np.ndarray:
    theta_y = math.atan2(target_point[0], target_point[2]) - math.atan2(point[0], point[2])
    R_y = np.array([
        [np.cos(theta_y), 0, np.sin(theta_y)],
        [0, 1, 0],
        [-np.sin(theta_y), 0, np.cos(theta_y)],
    ])
    
    return R_y

In [18]:
def rotate_along_z(point: np.ndarray, target_point: np.ndarray) -> np.ndarray:
    theta_z = math.atan2(target_point[1], target_point[0]) - math.atan2(point[1], point[0])
    R_z = np.array([
        [np.cos(theta_z), -np.sin(theta_z), 0],
        [np.sin(theta_z), np.cos(theta_z), 0],
        [0, 0, 1],
    ])
    
    return R_z

In [19]:
# R_y = rotate_along_y(point, target_point)
# next_point = R_y @ point
# next_point

In [20]:
# R_z = rotate_along_z(next_point, target_point)
# next_point = R_z @ next_point
# next_point

In [21]:
R_z = rotate_along_z(point, target_point)
next_point = R_z @ point
next_point

array([ 2.64765230e-17, -1.96197086e-01,  1.00000000e+00])

In [22]:
R_x = rotate_along_x(next_point, target_point)
next_point = R_x @ next_point
next_point

array([ 2.64765230e-17, -1.01906491e+00,  1.11022302e-16])

In [23]:
next_point[abs(next_point) < 1e-13] = 0.0
next_point

array([ 0.        , -1.01906491,  0.        ])

In [24]:
H = K @ R_x @ R_z @ np.linalg.inv(K)
H

array([[ 6.14631788e-01,  8.81077254e-01, -2.34572843e+03],
       [-3.24352031e-01,  4.86899791e-01, -4.36185103e+03],
       [-1.20896648e-04,  1.81483533e-04,  8.34701895e-02]])

In [25]:
# H = K @ R_z @ R_y @ np.linalg.inv(K)
# H

In [30]:
output_image = cv2.warpPerspective(image, H, (image.shape[1] * 3, image.shape[0] * 3))

In [31]:
show_image("OutputImage", output_image)